# configs

## rtst.yaml

``` yaml
device: auto   # 可选值：auto、cuda、cpu
experiment: 4 # 训练实验号，影响部分文件名字，比如日志文件，Tensorboard文件
load: 3 # 加载权重标号，前面是RTST_，下一个是保存的标号
save: 3
a: 1 #1 # 三种损失的比例
b: 100000 #1e5
c: 1e-7 #5e-6
style_pic: 'data/impression.jpg' # 风格图片路径
content_layers: ['15']               # relu3_3
style_layers: ['3', '8', '15', '22']  # relu1_2, relu2_2, relu3_3, relu4_3
weights_path: 'weights/' # 权重文件夹
mean: [0.485, 0.456, 0.406] # imagenet常用数值
std: [0.229, 0.224, 0.225]
pic_size: 256 # 输入图片裁切尺寸

lr: 1e-3            # 学习率  
epochs: 20          # 迭代次数  
batch_size: 16
freq: 50 # 多少步打印一个Batchsize的图片
train_subset_len: 20000       # 划分数据集子集大小              
val_subset_len: 1000                    
mode: 'train' # 模式，test是使用data下的test文件夹中的一个图片，对一个图片做过拟合尝试，还有一些参数都会在src/utils/cfg.py中自动做修改；train是子集下训练；full_train是在完整数据集下训练

cv_mode: 'video' # video是处理视频模式，调用visualize.py；camera是使用摄像头，但是wsl不支持
video_path: 'data/video/hfut.mp4' # 原视频路径
output_path: 'results/video/output_2.mp4' # 输出视频路径
weight_path: 'weights/rtst_4.pth' # 加载权重文件的路径
```

# data

train放置训练集数据，test中放置一张图片做简单测试，val放置验证集数据，其他图片是风格图片

## StyleDataset.py

根据参数文件中的值设定不同的数据集

# models

## rtst.py

架构是直接对图片处理，不将一份图片残差传到最终输出。

## results

store中放置训练过程中可视化出来的中间结果。video放置实时处理视频的结果。

# src

## fe_model.py

提取图片特征模型。

## test.py

测试wsl中OpenCV的使用。

## visualize.py

In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from models.rtst import TransformerNet
from src.utils.cfg import cfg, transform_pic, transform_pics

cv_mode = cfg['cv_mode']
mean = cfg['mean']
std = cfg['mean']


# 初始化参数设备和模型
device = cfg['device']
video_path = cfg['video_path']
weight_path = cfg['weight_path']
output_path = cfg['output_path']

print(device)
model = TransformerNet().to(device).eval()
checkpoint = torch.load(weight_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint)

# 后处理：(-1,1)->(0,1) 并转回 NumPy
def postprocess(tensor):
    img = (tensor + 1) / 2
    img = img.clamp(0, 1)
    img = img.permute(1, 2, 0).cpu().numpy()  # H×W×C
    return (img * 255).astype(np.uint8)

# 打开视频文件

if cv_mode == 'video':
    # 处理视频
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')        # mp4v/mjpg/xvid
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    delay = int(1000 / fps)
elif cv_mode == 'camera':
    # 使用摄像头
    cap = cv2.VideoCapture(0)
    delay = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 预处理：BGR->RGB->PIL->Tensor
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    input_tensor = transform_pics(pil).unsqueeze(0).to(device)

    # 模型推理
    with torch.no_grad():
        output_tensor = model(input_tensor)[0]

    # 后处理 + 转回 BGR
    out_img = postprocess(output_tensor)
    out_bgr = cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR)

    out.write(out_bgr)

    # 显示与退出判断
    cv2.imshow("Stylized", out_bgr)
    key = cv2.waitKey(delay=delay)
    if key == ord('q'):
        break


cap.release()
out.release()
cv2.destroyAllWindows()

opencv做的就是读取视频文件然后一帧帧获取图片，传给模型然后显示和保存，中间需要定义一下对象以及对图片数据进行处理，转化。